In [1]:
from bs4 import BeautifulSoup
# basic
import numpy as np
import pandas as pd
import pymongo
# visual
import matplotlib.pyplot as plt
%matplotlib inline
#time
import datetime 

#talib
import talib
from talib import abstract

#gui

import requests
import re
import io

In [2]:
def crawl_price(stock_id):
    now = int(datetime.datetime.now().timestamp())+86400
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=0&period2=" + str(now) + "&interval=1d&events=history&crumb=hP2rOschxO0"

    response = requests.post(url)
    
    f = io.StringIO(response.text)
    df = pd.read_csv(f, index_col='Date', parse_dates=['Date'] )
    return df

In [3]:
def get_webservice(url):
    list_req = requests.get(url)
    soup = BeautifulSoup(list_req.content)
    r = soup.find('string').text
    return r

In [4]:
def putOrder(bsaction, stockid,  ordertype, volume, price):
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/PutOrderXML3?GMRIDStr=BCSP&CompCode="+stockid+"&Price="+price+"&Volume="+volume+"&BSAction="+bsaction+"&OrderType="+ordertype+"&IsOddLot=0&Currency=TWD&OrderNote=ROD&OCType=0&CombineNo=&OrderParameter=0&Lang=TC&str_ip=127.0.0.1"
        client = get_webservice(url)
        
        """try:
            import xml.etree.cElementTree as ET
        except ImportError:
            import xml.etree.ElementTree as ET
        r = ET.fromstring(client)
        """
        r = client.split("\"")
        if r[1] == "Success":
            return "委託下單成功"
            #tk.messagebox.showinfo(title='回報', message='Success')
        elif r[1] == "Failure":
            return "委託下單失敗"+'原因:'+r[5]
            #tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:'+new[5])
        else:
            return "委託下單失敗"+'原因:格式輸入錯誤'
            #tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:格式輸入錯誤')

In [43]:
#9日 K值低於 20% 買進，高於 80% 賣出
def testKD92080set(start_money,close,data):
    K = data.iloc[0:,1] #取 K 值
    #D = data.iloc[0:,2] #取 D 值
    #DIF = data2.iloc[0:,0] #取 DIF 值
    #DEA = data2.iloc[0:,1] #取 DEA 值
    #MACD = data2.iloc[0:,2] #取 MACD 值
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    principal = float(start_money)
    stock_count = 0 #股票張數
    bought_money = 0 #剩餘金額
    final_money = 0 #最終金額
    profit = 0 #獲利
    rateOfReturn = 0.0 #報酬率
    record = [] #存取購買股票時的股價
    lastROP = [] #存取報酬率

    try:
        
        for i in range(len(K)):
            print(date[i])
            print('K值為:',K[i])
            print(i)
            print(close[i])
            if(stock_count==0):
                if(float(start_money)>=close[i]*1000):
                    if(K[i]<20):
                        print('K值為:',K[i])
                        print("買了一張股票，股價為:",close[i])
                        if(date[i]== str(datetime.date.today())+" 00:00:00"):
                                #putOrder("B", "2330.TW",  "MKT", "1000", "0")
                            print("2019-08-02 成功下單")
                        else:
                            stock_count = stock_count + 1
                            start_money = float(start_money) - close[i]*1000
                            record.append(close[i]) #將買的股價放入recorde陣列裡
                            print(record)
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')

                        #"買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額" 
                    else:
                        print("尚未達到買點")
                        #continue
                else:
                    print("錢不購買")
                    #continue
            else:
                if(close[i] >= record[-1]+5): #現在的股價 >= 停利條件
                    print('現在的股價=',close[i])
                    print('條件: 現股>=最後一次購買股價+10 = ') # recode[-1]為最後一次交易的股價
                    print('最後一次購買股價',record[-1])
                    print('最後一次購買股價+5',record[-1]+5)
                    print('已達停利標準')
                    if(date[i]== str(datetime.date.today())+" 00:00:00"):
                        #putOrder("S", "2330.TW",  "MKT", "1000", "0")
                        print("2019-08-02 成功下單")
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)

                    lastROP.append(rateOfReturn)
                    start_money = final_money #將初始金額改為賣出後的總金額
                    #continue
                elif(close[i] <= record[-1]-5):
                    print('現在的股價=',close[i]) 
                    print('條件: 現股<=最後一次購買股價-5 = ')
                    print('最後一次購買股價',record[-1])
                    print('最後一次購買股價-5',record[-1]-5)
                    print('已達停損標準')
                    if(date[i][0:8]== "2019-08-02"): #str(datetime.date.today())+" 00:00:00"):
                            #putOrder("B", "2330.TW",  "MKT", "1000", "0")
                        print("2019-08-02 成功下單")
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("i:",i)
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)

                    lastROP.append(rateOfReturn)
                    start_money = final_money #將初始金額改為賣出後的總金額
                    #continue
                elif(K[i]<20):
                    if(float(start_money)>=close[i]*1000):
                        print('K值為:',K[i])
                        print("買了一張股票，股價為:",close[i])
                        if(date[i]== str(datetime.date.today())+" 00:00:00"):
                            #putOrder("B", "2330.TW",  "MKT", "1000", "0")
                            print("2019-08-02 成功下單")
                        stock_count = stock_count + 1
                        start_money = float(start_money) - close[i]*1000
                        record.append(close[i]) #將買的股價放入recorde陣列裡
                        print(record)
                        print('剩餘資金為:',start_money)
                        print('擁有'+str(stock_count)+ '張股票')

                        #"買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額" 
                    else:
                        print("錢不夠再買一張股票")
                        #continue
                elif(K[i]>80):
                    print('K值為:',K[i])
                    print("手上有"+str(stock_count)+"張股票可賣")
                    final_money = start_money + stock_count*close[i]*1000
                    if(date[i]== str(datetime.date.today())+" 00:00:00"):
                        #putOrder("S", "2330.TW",  "MKT", "1000", "0")
                        print("2019-08-02 成功下單")
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("i:",i)
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)


                    lastROP.append(rateOfReturn)
                    start_money = final_money
                else:
                    print("尚未達到買賣點")
                        #continue
                print("\n")        
                print('總報酬率',lastROP[-1])

                return lastROP[-1]
    except:
        return "Fail"

In [44]:
def first(stock,start_money,start_time,end_time):
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close, 2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;
    
    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
        #data.plot()
    data.reset_index(inplace=True)
        #print(data)

        #testKDcrossSet(start_money,close,sProfit,sLoss,data)

    return testKD92080set(start_money,close2,data)


In [45]:
first("2330.TW","10000000","2010-01-01",str(datetime.date.today()))

2010-01-04 00:00:00
K值為: nan
0
64.9
尚未達到買點
2010-01-05 00:00:00
K值為: nan
1
64.5
尚未達到買點
2010-01-06 00:00:00
K值為: nan
2
64.9
尚未達到買點
2010-01-07 00:00:00
K值為: nan
3
64.2
尚未達到買點
2010-01-08 00:00:00
K值為: nan
4
64.0
尚未達到買點
2010-01-11 00:00:00
K值為: nan
5
64.5
尚未達到買點
2010-01-12 00:00:00
K值為: nan
6
63.6
尚未達到買點
2010-01-13 00:00:00
K值為: nan
7
62.8
尚未達到買點
2010-01-14 00:00:00
K值為: nan
8
63.2
尚未達到買點
2010-01-15 00:00:00
K值為: nan
9
63.5
尚未達到買點
2010-01-18 00:00:00
K值為: nan
10
62.9
尚未達到買點
2010-01-19 00:00:00
K值為: nan
11
62.5
尚未達到買點
2010-01-20 00:00:00
K值為: 21.21651925200991
12
63.0
尚未達到買點
2010-01-21 00:00:00
K值為: 23.94632448146204
13
62.6
尚未達到買點
2010-01-22 00:00:00
K值為: 19.371140898236686
14
61.1
K值為: 19.371140898236686
買了一張股票，股價為: 61.1
[61.1]
剩餘資金為: 9938900.0
擁有1張股票
2010-01-25 00:00:00
K值為: 10.734076904119915
15
60.4
K值為: 10.734076904119915
買了一張股票，股價為: 60.4
[61.1, 60.4]
剩餘資金為: 9878500.0
擁有2張股票




'Fail'